# Review Data Inspection

Contains full review text data including the user_id that wrote the review and the business_id the review is written for.

In [ ]:
review_df = pd.read_feather('data/yelp_review.feather')
print(f"review data shape: {review_df.shape}")

In [ ]:
review_df.info()

In [ ]:
review_df.head(3)